<a href="https://colab.research.google.com/github/KennedyMen/Quickscripts/blob/main/Whisperx_Subs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!/bin/bash
!pip install git+https://github.com/m-bain/whisperx.git
!@pip install torch torchaudio
!pip install ffmpeg-python

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-yyuk5263
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-yyuk5263
  Resolved https://github.com/m-bain/whisperx.git to commit 44e8bf5bb658ad6e1a80585b987fae2d48b9662c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.

/bin/bash: line 1: @pip: command not found
ERROR: Operation cancelled by user


In [ ]:
import os
import whisperx
# Set the language for transcription
language = "fr"  # Change this to your desired language code (e.g., "es" for Spanish, "fr" for French)
# Load WhisperX model
device = "cuda"  # Use GPU for faster processing
model = whisperx.load_model("large-v2", device=device, language=language)


# Path to the directory containing audio files
audio_dir = "/content/drive/MyDrive/Colab Notebooks/Files/Media"

# Output directory for SRT files
output_dir = "/content/drive/MyDrive/Colab Notebooks/Files/Subs"
os.makedirs(output_dir, exist_ok=True)

# Function to convert seconds to SRT time format (HH:MM:SS,mmm)
def to_srt_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{int(seconds):02},{milliseconds:03}"

# Function to generate SRT content from WhisperX result
def generate_srt(result):
    srt_content = ""
    for i, segment in enumerate(result["segments"]):
        start_time = to_srt_time(segment["start"])
        end_time = to_srt_time(segment["end"])
        text = segment["text"].strip()
        srt_content += f"{i+1}\n{start_time} --> {end_time}\n{text}\n\n"
    return srt_content

# Iterate over each audio file in the directory
for audio_file in os.listdir(audio_dir):
    if audio_file.endswith(".wav") or audio_file.endswith(".mp3"):
        audio_path = os.path.join(audio_dir, audio_file)
        print(f"Transcribing {audio_file}...")

        # Transcribe the audio with the specified language
        result = model.transcribe(audio_path, language=language)  # Pass the language parameter here

        # Align timestamps for word-level accuracy
        model_a, metadata = whisperx.load_align_model(language_code=language, device=device)
        result = whisperx.align(result["segments"], model_a, metadata, audio_path, device)

        # Generate SRT content
        srt_content = generate_srt(result)

        # Save the SRT file
        output_file = os.path.join(output_dir, f"{os.path.splitext(audio_file)[0]}.srt")
        with open(output_file, "w") as f:
            f.write(srt_content)

        print(f"SRT file saved to {output_file}")

print("All transcriptions completed!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

>>Performing voice activity detection using Pyannote...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.
Transcribing HPI.S01E04.mp3...


/usr/local/lib/python3.11/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
